## Specialization

The first time a newly defined funciton is executed, it will be compiled to native code and executed.  On subsequent calls that native compiled code is executed without the overhead of the compilation process.

You can clearly observe this with the funciton defined below.

In [1]:
quadratic(x) = x.^2 .+ 2x .+ 5

quadratic (generic function with 1 method)

To provide some work for the compiler, you can call the funciton with a vector.

In [2]:
x = rand(10_000);

In [3]:
@time quadratic(x);
@time quadratic(x);

  0.203708 seconds (720.81 k allocations: 37.400 MiB, 40.49% gc time, 99.96% compilation time)
  0.000051 seconds (4 allocations: 156.344 KiB)


The first call indeed takes orders of magnitude longer than the second or any subsequent calls for that matter.

However, not that when we call the function with an argument of a different type, it will be compiled again.

In [4]:
x = rand(Int, (10_000, ));

In [5]:
@time quadratic(x);
@time quadratic(x);

  0.137798 seconds (623.40 k allocations: 31.789 MiB, 99.94% compilation time)
  0.000040 seconds (4 allocations: 156.344 KiB)


You can use the `MethodAnalysis` package to gain insight into the instancees that Julia generated for your funcitons by calling the `methodinstances` function.

In [6]:
using MethodAnalysis

In [9]:
methods(quadratic)

# 1 method for generic function "quadratic":
[1] quadratic(x) in Main at In[1]:1

In [10]:
methodinstances(quadratic)

2-element Vector{Core.MethodInstance}:
 MethodInstance for quadratic(::Vector{Float64})
 MethodInstance for quadratic(::Vector{Int64})

Clearly, two instances of the function have been produced, one for a `Vector` of `Float64`, the other for `Int64` elements.

When you call the function with a scalar floating point value, a third instance is created, but given the fact that this is a very simple function, the compilation hardly takes any time at all.

In [11]:
@time quadratic(3.0);
@time quadratic(12.0);

  0.000001 seconds
  0.000001 seconds


In [12]:
methodinstances(quadratic)

3-element Vector{Core.MethodInstance}:
 MethodInstance for quadratic(::Vector{Float64})
 MethodInstance for quadratic(::Vector{Int64})
 MethodInstance for quadratic(::Float64)

## Just Ahead Of Time compilation

When you call a function for the first time, the source code is converted into native machine code in a process with many steps.  The result of these steps can be inspected using macros.

To illustrate this, you can define another function.

In [1]:
axpy(alpha, x, y) = alpha*x + y

axpy (generic function with 1 method)

One of the first steps you can inspect is the result of macro expansion, e.g., what happens when you use the `@time` macro.

In [19]:
@macroexpand @time axpy(2.0, 3.0, 5.0)

quote
    #= timing.jl:253 =#
    begin
        #= timing.jl:258 =#
        $(Expr(:meta, :force_compile))
        #= timing.jl:259 =#
        local var"#86#stats" = Base.gc_num()
        #= timing.jl:260 =#
        local var"#88#elapsedtime" = Base.time_ns()
        #= timing.jl:261 =#
        Base.cumulative_compile_timing(true)
        #= timing.jl:262 =#
        local var"#89#compile_elapsedtimes" = Base.cumulative_compile_time_ns()
        #= timing.jl:263 =#
        local var"#87#val" = $(Expr(:tryfinally, :(axpy(2.0, 3.0, 5.0)), quote
    var"#88#elapsedtime" = Base.time_ns() - var"#88#elapsedtime"
    #= timing.jl:265 =#
    Base.cumulative_compile_timing(false)
    #= timing.jl:266 =#
    var"#89#compile_elapsedtimes" = Base.cumulative_compile_time_ns() .- var"#89#compile_elapsedtimes"
end))
        #= timing.jl:268 =#
        local var"#90#diff" = Base.GC_Diff(Base.gc_num(), var"#86#stats")
        #= timing.jl:269 =#
        local var"#91#_msg" = Base.nothing
        #= timi

As you can see, a lot of bookkeeping is required to take track of the time and number of allocation accurately,  After writing the timing results, the result of the Julia expression is returned.

This function is of course most useful when you develop your own macros.

The code is subsequently converted into an internal representation, i.e., the code is "lowered".

In [26]:
@code_lowered axpy(2.0, 3.0, 5.0)

CodeInfo(
1 ─ %1 = alpha * x
│   %2 = %1 + y
└──      return %2
)

In the next phase, types are derived for this code, which you can inspect as well.

In [27]:
@code_typed axpy(2.0, 3.0, 5.0)

CodeInfo(
1 ─ %1 = Base.mul_float(alpha, x)::Float64
│   %2 = Base.add_float(%1, y)::Float64
└──      return %2
) => Float64

Since all function arguments are `Float64` in this case, all intermediate types as well as the return type of the funciton will be `Float64` as well.

On the other hand, if the first two arguments are `Int`, you observe a different intermediate type, as well as some type conversions.

In [28]:
@code_typed axpy(2, 3, 5.0)

CodeInfo(
1 ─ %1 = Base.mul_int(alpha, x)::Int64
│   %2 = Base.sitofp(Float64, %1)::Float64
│   %3 = Base.add_float(%2, y)::Float64
└──      return %3
) => Float64

Obviously, when the function is called with arguments that are all `Int`, the result type of the function will be `Int64`.

In a subsequence phase, the intermediate code is translated to LLVM representation, the virtual machine code used by the underlying compiler.

In [29]:
@code_llvm axpy(2.0, 3.0, 5.0)

;  @ In[13]:1 within `axpy`
define double @julia_axpy_2860(double %0, double %1, double %2) #0 {
top:
; ┌ @ float.jl:385 within `*`
   %3 = fmul double %0, %1
; └
; ┌ @ float.jl:383 within `+`
   %4 = fadd double %3, %2
; └
  ret double %4
}


Finally, native code is produced in which you recognize the familiar machine instructions.

In [30]:
@code_native axpy(2.0, 3.0, 5.0)

	.text
	.file	"axpy"
	.globl	julia_axpy_2894                 # -- Begin function julia_axpy_2894
	.p2align	4, 0x90
	.type	julia_axpy_2894,@function
julia_axpy_2894:                        # @julia_axpy_2894
; ┌ @ In[13]:1 within `axpy`
	.cfi_startproc
# %bb.0:                                # %top
; │┌ @ float.jl:385 within `*`
	vmulsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ float.jl:383 within `+`
	vaddsd	%xmm2, %xmm0, %xmm0
; │└
	retq
.Lfunc_end0:
	.size	julia_axpy_2894, .Lfunc_end0-julia_axpy_2894
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


## Type stability



The performance of your code can suffer significantly when type instability occurs.  This happens when the return type of a function is ambiguous.  As a trivial example, consider calling the function `distance` with a vector of floating point values, and a vector for which each element is type `Any`.

In [4]:
distance(v1, v2) = sqrt((v1[1] - v2[1])^2 + (v1[2] - v2[2])^2)

distance (generic function with 1 method)

In [5]:
using BenchmarkTools

In [11]:
@btime distance(v1, v2) setup=(v1=rand(2); v2=rand(2));
@btime distance(v1, v2) setup=(v1=Any[rand(), rand()]; v2=Any[rand(), rand()]);

  1.884 ns (0 allocations: 0 bytes)
  74.452 ns (6 allocations: 96 bytes)


The difference in performance between the two computation is substantial.

Fortunately, the `@code_warntype` macro can be used to check for this.  It will print declarations that lead to type stability in red.

In [ ]:
@code_warntype distance(rand(2), rand(2))

MethodInstance for distance(::Vector{Float64}, ::Vector{Float64})
  from distance(v1, v2) in Main at In[4]:1
Arguments
  #self#::Core.Const(distance)
  v1::Vector{Float64}
  v2::Vector{Float64}
Body::Float64
1 ─ %1  = Base.getindex(v1, 1)::Float64
│   %2  = Base.getindex(v2, 1)::Float64
│   %3  = (%1 - %2)::Float64
│   %4  = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %5  = (%4)()::Core.Const(Val{2}())
│   %6  = Base.literal_pow(Main.:^, %3, %5)::Float64
│   %7  = Base.getindex(v1, 2)::Float64
│   %8  = Base.getindex(v2, 2)::Float64
│   %9  = (%7 - %8)::Float64
│   %10 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %11 = (%10)()::Core.Const(Val{2}())
│   %12 = Base.literal_pow(Main.:^, %9, %11)::Float64
│   %13 = (%6 + %12)::Float64
│   %14 = Main.sqrt(%13)::Float64
└──       return %14



In [12]:
@code_warntype distance(Any[rand(), rand()], Any[rand(), rand()])

MethodInstance for distance(::Vector{Any}, ::Vector{Any})
  from distance(v1, v2) in Main at In[4]:1
Arguments
  #self#::Core.Const(distance)
  v1::Vector{Any}
  v2::Vector{Any}
Body::Any
1 ─ %1  = Base.getindex(v1, 1)::Any
│   %2  = Base.getindex(v2, 1)::Any
│   %3  = (%1 - %2)::Any
│   %4  = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %5  = (%4)()::Core.Const(Val{2}())
│   %6  = Base.literal_pow(Main.:^, %3, %5)::Any
│   %7  = Base.getindex(v1, 2)::Any
│   %8  = Base.getindex(v2, 2)::Any
│   %9  = (%7 - %8)::Any
│   %10 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %11 = (%10)()::Core.Const(Val{2}())
│   %12 = Base.literal_pow(Main.:^, %9, %11)::Any
│   %13 = (%6 + %12)::Any
│   %14 = Main.sqrt(%13)::Any
└──       return %14



The example above is of course contrived, a more realistic example is given below.  The function simply computes the sum of the elements of a vector.

In [10]:
sum_all(v) = begin
    total = 0
    for value in v
        total += value
    end
    return total
end

sum_all (generic function with 1 method)

When you call the function with a vector of floating point values, the type of the variable `total` can not be determined unambiguously.  It is initialized with `0`, an `Int64` value.  Depending on whether the length of the vector is non-zero, floating point values are added to it.  At the return statement, `total` may be eitehr `Int64` or `Float64`.  This is indicated by the `Union` type.

In [11]:
v = rand(1000);
@code_warntype sum_all(v)

MethodInstance for sum_all(::Vector{Float64})
  from sum_all(v) in Main at In[10]:1
Arguments
  #self#::Core.Const(sum_all)
  v::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  total::Union{Float64, Int64}
  value::Float64
Body::Union{Float64, Int64}
1 ─       (total = 0)
│   %2  = v::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (value = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (total = total + value)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return total



This can be easily remedied by making sure that `total` is initialized by a zero value of a type that corresponds to the elements of the vector `v`), the argument of the function.

In [1]:
sum_all_2(v) = begin
    total = zero(eltype(v))
    for value in v
        total += value
    end
    return total
end

sum_all_2 (generic function with 1 method)

When calling `sum_all_2` with a vector containing `Float64` values, `total` will be initialized to a `Float64` 0.0 and its type is unambiguous.

In [13]:
v = rand(1000);
@code_warntype sum_all_2(v)

MethodInstance for sum_all_2(::Vector{Float64})
  from sum_all_2(v) in Main at In[12]:1
Arguments
  #self#::Core.Const(sum_all_2)
  v::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  total::Float64
  value::Float64
Body::Float64
1 ─ %1  = Main.eltype(v)::Core.Const(Float64)
│         (total = Main.zero(%1))
│   %3  = v::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}
│         (value = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (total = total + value)
│         (@_3 = Base.iterate(%3, %10))
│   %13 = (@_3 === nothing)::Bool
│   %14 = Base.not_int(%13)::Bool
└──       goto #4 if not %14
3 ─       goto #2
4 ┄       return total



Similarly, when the function `sum_all_2` is called with a vector of `Int32`, the code that is generated initializes `total` to an `Int32` 0 and again, its type is unambiguous.

In [2]:
v = rand(Int32, 1000);
@code_warntype sum_all_2(v)

MethodInstance for sum_all_2(::Vector{Int32})
  from sum_all_2(v) in Main at In[1]:1
Arguments
  #self#::Core.Const(sum_all_2)
  v::Vector{Int32}
Locals
  @_3::Union{Nothing, Tuple{Int32, Int64}}
  total::Int32
  value::Int32
Body::Int32
1 ─ %1  = Main.eltype(v)::Core.Const(Int32)
│         (total = Main.zero(%1))
│   %3  = v::Vector{Int32}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Int32, Int64}
│         (value = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (total = total + value)
│         (@_3 = Base.iterate(%3, %10))
│   %13 = (@_3 === nothing)::Bool
│   %14 = Base.not_int(%13)::Bool
└──       goto #4 if not %14
3 ─       goto #2
4 ┄       return total



Some care is required to ensure type stability, so the JAOT compilation can generate efficient code.